# Tested for `networks_2022`

In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import h5py
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from predify.utils.training import train_pcoders, eval_pcoders

from networks_2022 import BranchedNetwork

In [2]:
engram_dir = '/mnt/smb/locker/issa-locker/users/Erica/'

In [3]:
from pbranchednetwork_all import PBranchedNetwork_AllSeparateHP
PNetClass = PBranchedNetwork_AllSeparateHP
pnet_name = 'all'
p_layers = 'All Layers'

In [4]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')

checkpoints_dir = f'{engram_dir}hcnn/checkpoints/'
tensorboard_dir = f'{engram_dir}hcnn/tensorboard/'

Device: cuda:0


# PNet outputs

In [5]:
net = BranchedNetwork()
net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))

/share/issa/users/es3773/temp-hallucnn/src/models/layers.py:78: UserWarning: Inconsistent tf pad calculation in ConvLayer.
  warnings.warn('Inconsistent tf pad calculation in ConvLayer.')
/share/issa/users/es3773/temp-hallucnn/src/models/layers.py:173: UserWarning: Inconsistent tf pad calculation: 0, 1
  warnings.warn(f'Inconsistent tf pad calculation: {pad_left}, {pad_right}')


<All keys matched successfully>

In [6]:
pnet = PNetClass(net, build_graph=True)
pnet.load_state_dict(torch.load(
    f"{checkpoints_dir}{pnet_name}/{pnet_name}-50-regular.pth",
    map_location='cpu'
    ))
pnet.to(DEVICE)
pnet.build_graph = False        # only required for training
pnet.eval()

PBranchedNetwork_AllSeparateHP(
  (backbone): BranchedNetwork(
    (speech_branch): Sequential(
      (conv1): ConvLayer(
        (block): Sequential(
          (0): Conv2d(1, 96, kernel_size=(6, 14), stride=(3, 3), padding=(2, 6))
          (1): ReLU()
        )
      )
      (rnorm1): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool1): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv2): ConvLayer(
        (block): Sequential(
          (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2), padding=(1, 2))
          (1): ReLU()
        )
      )
      (rnorm2): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool2): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv3): ConvLayer(
        (block): Sequential(
          (0): Conv2d(256, 512, kernel_size=

In [7]:
y_hats = []

f_in = h5py.File(f"{engram_dir}PsychophysicsWord2017W_not_resampled.hdf5", 'r')
for ind in range(len(f_in['data'])):
    pnet.reset()
    _input = torch.tensor(np.reshape(f_in['data'][ind],(164,400))).to(DEVICE)
    word, genre = pnet(_input)
    for _ in range(3):
        word, genre = pnet(None)

    y_hat = int(torch.argmax(word))
    y_hats.append(y_hat)
    if len(y_hats) > 20:
        break

/share/issa/users/es3773/predify/predify/modules/base.py:260: UserWarning: Using a target size (torch.Size([164, 400])) that is different to the input size (torch.Size([1, 1, 164, 400])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  self.prediction_error  = nn.functional.mse_loss(self.prd, target)
/home/es3773/.local/lib/python3.7/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


In [8]:
# Torch network outputs
print(y_hats[:20])

[229, 312, 263, 3, 17, 5, 6, 7, 8, 9, 10, 11, 12, 13, 326, 108, 335, 298, 17, 11]


# tf Network Outputs

In [6]:
tf_y_hats = np.array([229, 312, 525, 3, 393, 5, 6, 7, 8, 9, 10, 11, 12, 13, 126, 108, 15, 298, 17, 46, 152, 388, 278, 459, 256, 23, 14, 25, 26, 27, 129, 29, 30, 31, 63, 337, 34, 216, 36, 37, 38, 241, 40, 41, 183, 42, 393, 240, 45, 196, 46, 487, 48, 129, 142, 51, 269, 519, 54, 55, 284, 57, 65, 58, 239, 59, 376, 62, 63, 158, 345, 66, 67, 68, 69, 70, 394, 72, 494, 74, 109, 75, 48, 78, 343, 455, 487, 413, 82, 109, 84, 496, 243, 490, 9, 24, 89, 0, 110, 259])

In [7]:
tf_y_hats_no_Dropout = [229, 312, 525, 3, 393, 5, 6, 7, 8, 9, 10, 11, 12, 13, 126, 108, 15, 298, 17, 46, 152, 388, 278, 459, 256, 23, 14, 25, 26, 27, 129, 29, 30, 31, 63, 337, 34, 216, 36, 37, 38, 241, 40, 41, 183, 42, 393, 240, 45, 196, 46, 487, 48, 129, 142, 51, 269, 519, 54, 55, 284, 57, 65, 58, 239, 59, 376, 62, 63, 158, 345, 66, 67, 68, 69, 70, 394, 72, 494, 74, 109, 75, 48, 78, 343, 455, 487, 413, 82, 109, 84, 496, 243, 490, 9, 24, 89, 0, 110, 259]

In [8]:
print(len(np.where(tf_y_hats == np.array(y_hats))[0]))

100


In [8]:
# http://localhost:1339/notebooks/JupyterNotebooks/thesis/analysesUsingNAC/net7_tf_2020w-naturalistic_heldOutSpkr.ipynb

# Think this is WSJ only -- 531 words in train set rather tha 587

# True Labels

In [38]:
f_in = h5py.File("../../data/small_cleansounds.hdf5", 'r')

In [39]:
y_hats = []
for ind in range(len(f_in['data'])):
    word, genre = net(torch.tensor(np.reshape(f_in['data'][ind], (164,400))))
    y_hat = int(torch.argmax(word))
    y_hats.append(y_hat)
    if len(y_hats) > 30:
        break

In [40]:
# Torch network outputs
print(y_hats[:20])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [32]:
# True labels
f_in['labels'][:20]

array([139, 457, 540, 403, 433, 403, 478, 362, 119, 259, 385, 235,  64,
       133, 457, 107, 561, 487, 487, 203], dtype=int32)

# PNet outputs over timesteps

In [9]:
net = BranchedNetwork()
net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))

<All keys matched successfully>

In [10]:
pnet = PNetClass(net, build_graph=True)
pnet.load_state_dict(torch.load(
    f"{checkpoints_dir}{pnet_name}/{pnet_name}-50-regular.pth",
    map_location='cpu'
    ))
pnet.to(DEVICE)
pnet.build_graph = False        # only required for training
pnet.eval()

PBranchedNetwork_AllSeparateHP(
  (backbone): BranchedNetwork(
    (speech_branch): Sequential(
      (conv1): ConvLayer(
        (block): Sequential(
          (0): Conv2d(1, 96, kernel_size=(6, 14), stride=(3, 3), padding=(2, 6))
          (1): ReLU()
        )
      )
      (rnorm1): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool1): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv2): ConvLayer(
        (block): Sequential(
          (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2), padding=(1, 2))
          (1): ReLU()
        )
      )
      (rnorm2): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool2): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv3): ConvLayer(
        (block): Sequential(
          (0): Conv2d(256, 512, kernel_size=

In [14]:
y_hats = []

f_in = h5py.File(f"{engram_dir}PsychophysicsWord2017W_not_resampled.hdf5", 'r')
for ind in range(len(f_in['data'])):
    pnet.reset()
    _input = torch.tensor(np.reshape(f_in['data'][ind],(164,400))).to(DEVICE)
    word, genre = pnet(_input)
    print(word[0,:20])
    print()
    for _ in range(3):
        word, genre = pnet(None)
        print(word[0,:20])
        print()

    y_hat = int(torch.argmax(word))
    y_hats.append(y_hat)
    if len(y_hats) > 20:
        break

tensor([-67.7944, -59.8435, -66.8536, -48.2975, -75.0216, -72.5112, -66.2891,
        -64.1743, -68.1044, -75.4633, -69.2653, -67.8985, -75.0611, -60.9579,
        -53.6056, -74.3391, -64.0351, -61.2078, -55.4650, -52.5376],
       device='cuda:0')

tensor([-62.5360, -55.4209, -62.1914, -44.9647, -69.6687, -67.6484, -61.2739,
        -59.7388, -62.7996, -69.8963, -64.2015, -63.0139, -69.2996, -56.4863,
        -49.6647, -68.7890, -59.1640, -56.8110, -50.8096, -48.3091],
       device='cuda:0')

tensor([-56.3348, -50.5037, -56.6246, -41.0962, -63.4249, -62.1353, -55.2719,
        -54.2770, -56.6699, -63.3887, -57.9580, -57.1879, -62.4397, -51.3251,
        -44.9899, -62.1404, -53.4112, -51.7251, -45.6808, -43.6086],
       device='cuda:0')

tensor([-50.8576, -46.0486, -51.3933, -37.6565, -57.6860, -57.1026, -49.7234,
        -49.1238, -51.1676, -57.1380, -51.9918, -51.8445, -55.9837, -46.5374,
        -40.7314, -55.9316, -48.1102, -46.9928, -41.1184, -39.4632],
       device='cuda:0')



tensor([-42.1778, -53.2476, -48.6881, -53.3334, -58.8775, -56.9210, -39.2189,
        -50.1079, -29.5500, -37.9707, -41.6753, -43.7829, -43.7799, -42.6146,
        -47.6389, -47.3284, -56.8736, -49.8580, -56.6996, -51.7103],
       device='cuda:0')

tensor([-39.5648, -50.0942, -45.5361, -49.8915, -55.0985, -53.6475, -36.5683,
        -46.8851, -27.6593, -35.3344, -38.6212, -40.7304, -40.8595, -39.7713,
        -44.6156, -43.7165, -53.3287, -46.6976, -53.0841, -48.8131],
       device='cuda:0')

tensor([-55.7495, -71.5822, -66.4896, -65.8686, -79.5732, -70.2889, -48.2585,
        -60.8659, -51.8088, -29.8886, -44.2662, -60.0271, -52.2519, -59.1306,
        -63.4626, -52.4044, -60.3970, -55.9917, -73.8818, -72.9757],
       device='cuda:0')

tensor([-52.9175, -67.2492, -62.7958, -61.6425, -75.1363, -66.4606, -44.4977,
        -55.8272, -48.0807, -27.4851, -41.2719, -55.9009, -48.3889, -55.4482,
        -59.6410, -48.9067, -56.5268, -52.4633, -69.7697, -68.8671],
       device='cuda:0')



tensor([-37.8825, -38.1628, -35.3373, -41.9410, -45.8253, -46.4349, -40.6861,
        -42.3663, -37.5044, -38.3845, -34.0130, -41.3844, -38.1246, -39.7641,
        -41.2842, -25.0304, -35.6899, -37.9645, -39.0748, -32.9422],
       device='cuda:0')

tensor([-21.2757, -24.5602, -20.9149, -22.3886, -24.5458, -23.9556, -24.9305,
        -25.8102, -23.7264, -27.5458, -24.6806, -22.6781, -21.7684, -22.0584,
        -18.2150, -22.3516, -16.5036, -19.7444, -25.2755, -22.3859],
       device='cuda:0')

tensor([-19.7531, -22.3822, -19.0396, -20.1361, -22.3708, -21.9701, -22.9328,
        -24.1116, -21.7943, -25.1495, -22.3496, -20.4971, -19.9713, -20.0686,
        -16.4368, -20.4844, -15.0311, -18.0416, -23.1275, -20.4699],
       device='cuda:0')

tensor([-18.0563, -19.9856, -17.0608, -17.5963, -20.0139, -19.7618, -20.7943,
        -22.1420, -19.6808, -22.4923, -19.9385, -18.2494, -18.0371, -17.7415,
        -14.5479, -18.6069, -13.4475, -16.0878, -20.6288, -18.5337],
       device='cuda:0')



In [15]:
net = BranchedNetwork(track_encoder_representations=True)
net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))
pnet = PNetClass(net, build_graph=True)
pnet.load_state_dict(torch.load(
    f"{checkpoints_dir}{pnet_name}/{pnet_name}-50-regular.pth",
    map_location='cpu'
    ))
pnet.to(DEVICE)
pnet.build_graph = False        # only required for training
pnet.eval()

PBranchedNetwork_AllSeparateHP(
  (backbone): BranchedNetwork(
    (speech_branch): Sequential(
      (conv1): ConvLayer(
        (block): Sequential(
          (0): Conv2d(1, 96, kernel_size=(6, 14), stride=(3, 3), padding=(2, 6))
          (1): ReLU()
        )
      )
      (rnorm1): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool1): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv2): ConvLayer(
        (block): Sequential(
          (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2), padding=(1, 2))
          (1): ReLU()
        )
      )
      (rnorm2): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool2): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv3): ConvLayer(
        (block): Sequential(
          (0): Conv2d(256, 512, kernel_size=

In [16]:
y_hats = []

f_in = h5py.File(f"{engram_dir}PsychophysicsWord2017W_not_resampled.hdf5", 'r')
for ind in range(len(f_in['data'])):
    pnet.reset()
    _input = torch.tensor(np.reshape(f_in['data'][ind],(164,400))).to(DEVICE)
    word, genre = pnet(_input)
    print(word[0,:20])
    print()
    for _ in range(3):
        word, genre = pnet(None)
        print(word[0,:20])
        print()

    y_hat = int(torch.argmax(word))
    y_hats.append(y_hat)
    if len(y_hats) > 20:
        break

tensor([-67.7944, -59.8435, -66.8536, -48.2975, -75.0216, -72.5112, -66.2891,
        -64.1743, -68.1044, -75.4633, -69.2653, -67.8985, -75.0611, -60.9579,
        -53.6056, -74.3391, -64.0351, -61.2078, -55.4650, -52.5376],
       device='cuda:0')

tensor([-62.5360, -55.4209, -62.1914, -44.9647, -69.6687, -67.6484, -61.2739,
        -59.7388, -62.7996, -69.8963, -64.2015, -63.0139, -69.2996, -56.4863,
        -49.6647, -68.7890, -59.1640, -56.8110, -50.8096, -48.3091],
       device='cuda:0')

tensor([-56.3348, -50.5037, -56.6246, -41.0962, -63.4249, -62.1353, -55.2719,
        -54.2770, -56.6699, -63.3887, -57.9580, -57.1879, -62.4397, -51.3251,
        -44.9899, -62.1404, -53.4112, -51.7251, -45.6808, -43.6086],
       device='cuda:0')

tensor([-50.8576, -46.0486, -51.3933, -37.6565, -57.6860, -57.1026, -49.7234,
        -49.1238, -51.1676, -57.1380, -51.9918, -51.8445, -55.9837, -46.5374,
        -40.7314, -55.9316, -48.1102, -46.9928, -41.1184, -39.4632],
       device='cuda:0')



tensor([-42.1778, -53.2476, -48.6881, -53.3334, -58.8775, -56.9210, -39.2189,
        -50.1079, -29.5500, -37.9707, -41.6753, -43.7829, -43.7799, -42.6146,
        -47.6389, -47.3284, -56.8736, -49.8580, -56.6996, -51.7103],
       device='cuda:0')

tensor([-39.5648, -50.0942, -45.5361, -49.8915, -55.0985, -53.6475, -36.5683,
        -46.8850, -27.6593, -35.3344, -38.6212, -40.7304, -40.8595, -39.7713,
        -44.6156, -43.7165, -53.3287, -46.6976, -53.0841, -48.8131],
       device='cuda:0')

tensor([-55.7495, -71.5822, -66.4896, -65.8686, -79.5732, -70.2889, -48.2585,
        -60.8659, -51.8088, -29.8886, -44.2662, -60.0271, -52.2519, -59.1306,
        -63.4626, -52.4044, -60.3970, -55.9917, -73.8818, -72.9757],
       device='cuda:0')

tensor([-52.9175, -67.2492, -62.7958, -61.6425, -75.1363, -66.4606, -44.4977,
        -55.8272, -48.0807, -27.4851, -41.2719, -55.9009, -48.3889, -55.4482,
        -59.6410, -48.9067, -56.5268, -52.4633, -69.7697, -68.8671],
       device='cuda:0')



tensor([-21.2757, -24.5602, -20.9149, -22.3886, -24.5458, -23.9556, -24.9305,
        -25.8102, -23.7264, -27.5458, -24.6806, -22.6781, -21.7684, -22.0584,
        -18.2150, -22.3516, -16.5036, -19.7444, -25.2755, -22.3859],
       device='cuda:0')

tensor([-19.7531, -22.3822, -19.0396, -20.1361, -22.3708, -21.9701, -22.9328,
        -24.1116, -21.7943, -25.1495, -22.3496, -20.4971, -19.9713, -20.0686,
        -16.4368, -20.4844, -15.0311, -18.0416, -23.1275, -20.4699],
       device='cuda:0')

tensor([-18.0563, -19.9856, -17.0608, -17.5963, -20.0139, -19.7618, -20.7943,
        -22.1420, -19.6808, -22.4923, -19.9385, -18.2494, -18.0371, -17.7415,
        -14.5479, -18.6069, -13.4475, -16.0878, -20.6288, -18.5337],
       device='cuda:0')

tensor([-16.4320, -17.7542, -15.2940, -15.3619, -17.9489, -17.6524, -18.7557,
        -20.1622, -17.7971, -20.0829, -17.8880, -16.2360, -16.1493, -15.5884,
        -12.8559, -16.8513, -12.0137, -14.2574, -18.3431, -16.8335],
       device='cuda:0')

